# Scratch book to try out different functions

## 1. Read lines in move_lines to build a dict of lines

In [ ]:
from data import TextReader

In [ ]:
file_name = "./Data/cornell_data/movie_lines.txt"
reader = TextReader()
reader.read_line_dict(file_name)

In [ ]:
# The first line stored in movie_lines.txt
print(reader.lines[1045])

## 2. Get vocabulary

In [ ]:
from collections import defaultdict
vocab = defaultdict(int)
for index in reader.lines.keys():
    line = reader.lines[index]
    words = line.split()
    for word in words:
        vocab[word] += 1

print(len(vocab))
print(vocab[b'They'])

## 3. Read dialogues

In [ ]:
# Default file path is movie_conversations
reader.read_dialogues()
print(reader.dialogues[0])

## 4. Sanity Test on Word2Vec Prep
(Not tested yet)

In [1]:
from word2vec import word2vec
model = word2vec()
model.add_to_sentence("I am good")
print(model.vocab)
print(model.word2idx)
print(model.idx2word)

NameError: name 'embedding_dims' is not defined